In [9]:
import os
import numpy as np
import time
import math
import random
from collections import deque

import torch
from torch_geometric.data import Dataset, Data

In [68]:
def RIS(adj_list, seed_idx, del_edge_num, eps=0.2):
    time_start = time.time()
    time_alg = []
    n = len(adj_list)

    edges = []
    adj_list_rev = [[] for _ in range(n)]
    for u in range(n):
        for v,p in adj_list[u]:
            edges.append([u,v,p])
            adj_list_rev[v].append((u,p))
    m = len(edges)

    rounds = int(0.1*n*math.log(n)/(eps**2))
    print(rounds)
    
    mask = []
    for _ in range(del_edge_num):
        count = np.zeros(m, dtype=int)
        for _ in range(rounds):
            active = np.zeros(n, dtype=int)
            active_adj_list_rev = [set() for _ in range(n)]
    
            # sample a terminal node
            terminal = random.randint(0,n-1)
            active[terminal] = 1
    
            # sample activated graph from the terminal node
            Q = deque([terminal])
            while Q:
                u = Q.popleft()
                for v,p in adj_list_rev[u]:
                    if active[v]: continue
                    if random.random()<p:
                        active[v]=1
                        Q.append(v)
                        active_adj_list_rev[u].add(v)
    
            # for each edge, check whether the terminal node is reachable from seeds without that edge
            if not any(active[seed_idx]): continue
            active = np.zeros(n, dtype=int)
            active[terminal] = 1
            for i,(s,t,p) in enumerate(edges):
                if s not in active_adj_list_rev[t]: count[i]+=1; continue
                Q = deque([terminal])
                while Q:
                    u = Q.popleft()
                    for v in active_adj_list_rev[u]:
                        if active[v] or (v,u)==(s,t): continue
                        active[v]=1
                        Q.append(v)
                if any(active[seed_idx]): count[i]+=1
            
        min_i = np.argmin(count)
        u,v,p = edges[min_i]
        mask.append([u,v,p])
        del edges[min_i]
        adj_list[u].remove((v,p))
        adj_list_rev[v].remove((u,p))
        m-=1
        
        time_alg.append(time.time()-time_start)
    return mask, time_alg